## Tensorflow の学習済みモデルを学習に利用する
### 学習済みモデルのダウンロード
学習済みモデルがS3にアップロードされていれば、学習インスタンスで読み込んで、学習済みモデルから学習をスタートすることできます。
まずは、tensorflowのresnet_v1_50のモデルをダウンロード・解凍します。

In [ ]:
import urllib.request
import os

download_file = "resnet_v1_50_2016_08_28.tar.gz"
if not os.path.exists(download_file):
    url = 'http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz'
    urllib.request.urlretrieve(url, download_file)
    
!tar xvzf resnet_v1_50_2016_08_28.tar.gz

### S3への学習済みモデルのアップロード
SageMaker Python SDKを利用して、ファイル resnet_v1_50.ckpt をS3にアップロードします。この学習済みモデルはtf.slimのモデルなので、Tensorflowのスクリプト(cifar100.py)ではtf.slimのモデルとして読み込まれるようにします。

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

bucket_name = sagemaker_session.default_bucket()
prefix_name  = 'pretrained_model/resnet_tf'
file_name = 'resnet_v1_50.ckpt'
model_file = sagemaker_session.upload_data(path=file_name, bucket=bucket_name, key_prefix=prefix_name)
print('Your pretrained model is uploaded to: {}'.format(model_file))

### 学習データのアップロード
- 今回はtf.kerasのデータセットからcifar100をダウンロードして利用します。
- `convert_to`の関数でtfrecord形式に変換して、アップロードします

In [ ]:
import os
import tensorflow as tf

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to(data_set, filename):
    """Converts a dataset to tfrecords."""
    images = data_set["images"]
    labels = data_set["labels"]
    num_examples = images.shape[0]

    rows = images.shape[1]
    cols = images.shape[2]
    if len(images.shape) > 3:
        depth = images.shape[3]
    else:
        depth = 1
        
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
train = {"images": x_train, "labels": y_train}
test= {"images": x_test, "labels": y_test}
os.makedirs("./data", exist_ok=True)
convert_to(train, "./data/train.tfrecords")
convert_to(test, "./data/test.tfrecords")
input_data = sagemaker_session.upload_data(path='./data', key_prefix='pretrained_model/data')
print('Your pretrained model is uploaded to: {}'.format(input_data))

## SageMakerでの学習

### Local mode用スクリプトダウンロード

デバッグでLocal Modeを実行するために、スクリプトをダウンロードして実行します。

In [ ]:
download_file = "setup.sh"
if not os.path.exists(download_file):
    url = 'https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker-python-sdk/tensorflow_distributed_mnist/setup.sh'
    urllib.request.urlretrieve(url, download_file)
    
!sh setup.sh

### 学習済みモデルに対する転移学習の実行
- S3にある学習済みモデルの場所をbucket_nameとprefix_nameで渡します。
- S3の学習データの場所はinput_dataとしてfitに渡します。
- `cifar100.py`内での読み込みは以下のとおりです。boto3でファイルをダウンロード、init_from_checkpointで読み込みます。 

(参考)
- https://github.com/tensorflow/tensorflow/issues/14713
- https://stackoverflow.com/questions/47867748/transfer-learning-with-tf-estimator-estimator-framework

```python
    bucket_name = params['bucket_name']
    prefix_name = params['prefix_name']
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(bucket_name).download_file(prefix_name, 'resnet.ckpt')
        print("Pretrained model is downloaded.")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise
            
            ...
    tf.train.init_from_checkpoint("./resnet.ckpt",{v.name.split(':')[0]: v for v in variables_to_restore if not 'biases' in v.name})
```

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='cifar100.py',
                             role=role,
                             framework_version='1.11.0',
                             training_steps=100, 
                             evaluation_steps=10,
                             train_instance_count=1,
                             train_instance_type='local',
                             hyperparameters={'bucket_name': bucket_name,
                                                                 'prefix_name': prefix_name +"/" + file_name})

mnist_estimator.fit(input_data)